In [14]:
from math import sqrt
from random import randint,shuffle
import random
import time

# Defining infinity
INF = 1e18
mult = 1e6
 
# Structure to represent a 2D point
class Point :
    def __init__(self,X=0,Y=0) -> None:
        self.X=X
        self.Y=Y
        
# Structure to represent a 2D circle
class Circle :
    def __init__(self,c=Point(),r=0) -> None:       
        self.C=c
        self.R=r
 
 
# Function to return the euclidean distance
# between two points
def dist(a, b):
    return sqrt(pow(a.X - b.X, 2)
                + pow(a.Y - b.Y, 2))

# Function to check whether a point lies inside
# or on the boundaries of the circle
def is_inside(c, p):
    return dist(c.C, p) <= c.R
 
# The following two functions are used
# To find the equation of the circle when
# three points are given.
 
# Helper method to get a circle defined by 3 points
def get_circle_center(bx, by,
                        cx, cy):
    B = bx * bx + by * by
    C = cx * cx + cy * cy
    D = bx * cy - by * cx
    return Point((cy * B - by * C) / (2 * D),
             (bx * C - cx * B) / (2 * D))
 
# Function to return the smallest circle
# that intersects 2 points
def circle_from1(A, B):
    # Set the center to be the midpoint of A and B
    C = Point((A.X + B.X) / 2.0, (A.Y + B.Y) / 2.0 )
 
    # Set the radius to be half the distance AB
    return Circle(C, dist(A, B) / 2.0 )
 
# Function to return a unique circle that
# intersects three points
def circle_from2(A, B, C):
    I = get_circle_center(B.X - A.X, B.Y - A.Y,
                                C.X - A.X, C.Y - A.Y)
 
    I.X += A.X
    I.Y += A.Y
    return Circle(I, dist(I, A))
 
 
 
 
 
# Function to check whether a circle
# encloses the given points
def is_valid_circle(c, P):
 
    # Iterating through all the points
    # to check  whether the points
    # lie inside the circle or not
    for p in P:
        if (not is_inside(c, p)):
            return False
    return True
 
# Function to return the minimum enclosing
# circle for N <= 3
def min_circle_trivial(P):
    assert(len(P) <= 3)
    if not P :
        return Circle()
     
    elif (len(P) == 1) :
        return Circle(P[0], 0)
     
    elif (len(P) == 2) :
        return circle_from1(P[0], P[1])
     
 
    # To check if MEC can be determined
    # by 2 points only
    for i in range(3):
        for j in range(i + 1,3):
 
            c = circle_from1(P[i], P[j])
            if (is_valid_circle(c, P)):
                return c
         
     
    return circle_from2(P[0], P[1], P[2])
 
# Returns the MEC using Welzl's algorithm
# Takes a set of input points P and a set R
# points on the circle boundary.
# n represents the number of points in P
# that are not yet processed.
def welzl_helper(P, R, n):
    # Base case when all points processed or |R| = 3
    if (n == 0 or len(R) == 3) :
        return min_circle_trivial(R)
     
 
    # Pick a random point randomly
    idx = randint(0,n-1)
    p = P[idx]
 
    # Put the picked point at the end of P
    # since it's more efficient than
    # deleting from the middle of the vector
    P[idx], P[n - 1]=P[n-1],P[idx]
 
    # Get the MEC circle d from the
    # set of points P - :p
    d = welzl_helper(P, R.copy(), n - 1)
 
    # If d contains p, return d
    if (is_inside(d, p)) :
        return d
     
 
    # Otherwise, must be on the boundary of the MEC
    R.append(p)
 
    # Return the MEC for P - :p and R U :p
    return welzl_helper(P, R.copy(), n - 1)

def minimum_enclosing_circle(P):
    P_copy = P.copy()
    shuffle(P_copy)
    return welzl_helper(P_copy, [], len(P_copy))

if __name__=='__main__':
    alt = 1000                     
    lat_position = 24.788090       
    lon_position = 120.997858      
    TxR = 300
    lat = [0]*300
    lon = [0]*300
    laspeed = [0]*300
    lospeed = [0]*300
    add_flag = [0]*300
    lospeed_total = 0
    laspeed_total = 0
    latmin = 24.7890
    latmax = 24.7895
    lonmin = 120.9975
    lonmax = 120.9980
    smax = 200*0.000009
    smin = 20*0.000009
    user = 100
    count_out = 0
    count_num = 0
    time_max = 0
    ladiff_total = 0
    lodiff_total = 0
    
    
    
    f= open("users_6G2.txt","r")
    while True:
        laspeed_total = 0
        lospeed_total = 0
        count_num += 1
        start = time.time()
        print("Current users number = ",user)
        for i in range(user):
            line = f.readline()
            lat[i] = float(line)
            line = f.readline()
            lon[i] = float(line)
            line = f.readline()
            laspeed[i] = float(line)
            line = f.readline()
            lospeed[i] = float(line)
            line = f.readline()
            add_flag[i] = float(line)
            
            
        v1 = [Point(lon[0]*mult,lat[0]*mult)]
        for i in range(1, user):
            v1.append(Point(lon[i]*mult,lat[i]*mult))
        mec = minimum_enclosing_circle(v1)
        
        for i in range(0,user):
            lospeed_total += lospeed[i];
            laspeed_total += laspeed[i];
        lospeed_uav = lospeed_total/user;
        laspeed_uav = laspeed_total/user;

        
        if mec.R/mult > TxR*0.000009:
            print("Out of range")
            for i in range(user-1 , -1, -1): # 99~0
                v2 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                for j in range(user-2 , i-1, -1): #user-i
                    v2.append(Point(lon[j]*mult,lat[j]*mult))
                mec2 = minimum_enclosing_circle(v2);
                if mec2.R/mult > TxR*0.000009:
                    print("Kicked off user below ",i)
                    v3 = [Point(lon[user-1]*mult,lat[user-1]*mult)]
                    for j in range(user-2 , i ,-1):
                        v3.append(Point(lon[j]*mult,lat[j]*mult))
                    mec3 = minimum_enclosing_circle(v3)
                    for m in range(i+1):
                        if sqrt(pow(lat[m] - mec3.C.Y/mult, 2) + pow(lon[m] - mec3.C.X/mult, 2)) > TxR * 0.000009:
                            print("Kick off user ",m)
                            lat[m] = 0
                    cnt = 0
                    for k in range(user):
                        if lat[k] != 0:
                            #print("lat[",cnt,"] = lat[",k,"]")
                            lat[cnt] = lat[k]
                            cnt += 1
                    user = cnt
                    if mec3.C.Y/mult > lat_position:
                        print("前移",(mec3.C.Y/mult - lat_position) / 0.000009,"公尺")
                    else:
                        print("後移",(lat_position - mec3.C.Y/mult) / 0.000009,"公尺")
                    if mec3.C.X/mult > lon_position:
                        print("左移",(mec3.C.X/mult - lon_position) / 0.000009,"公尺")
                    else:
                        print("右移",(lon_position - mec3.C.X/mult) / 0.000009,"公尺")
                    lat_position = mec3.C.Y/mult
                    lon_position = mec3.C.X/mult
                    break
        elif mec.R/mult == 0:
            print("mec.Y = %5.6f" %mec.C.Y/mult)
            print("mec.X = %5.6f" %mec.C.X/mult)
            if mec.C.Y/mult > lat_position:
                print("前移",(mec.C.Y/mult - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y/mult) / 0.000009,"公尺")
            if mec.C.X/mult > lon_position:
                print("左移",(mec.C.X/mult - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X/mult) / 0.000009,"公尺")
            
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
           
        else:
            print("lat = ",mec.C.Y/mult)
            print("lon = ",mec.C.X/mult)
            #print("R = ",mec.R/mult)
            if mec.C.Y > lat_position:
                print("前移",(mec.C.Y - lat_position) / 0.000009,"公尺")
            else:
                print("後移",(lat_position - mec.C.Y) / 0.000009,"公尺")
            if mec.C.X > lon_position:
                print("左移",(mec.C.X - lon_position) / 0.000009,"公尺")
            else:
                print("右移",(lon_position - mec.C.X) / 0.000009,"公尺")
            print("real laspeed = ",abs(lat_position - mec.C.Y/mult) / 0.000009/0.03,"m/s")
            print("real lospeed = ",abs(lon_position - mec.C.X/mult) / 0.000009/0.03,"m/s")
            print("predict laspeed = ",laspeed_uav/0.03,"m/s")
            print("predict lospeed = ",lospeed_uav/0.03,"m/s")
            larspd = abs((lat_position - mec.C.Y/mult) / 0.000009/0.03)
            lorspd = abs((lon_position - mec.C.X/mult) / 0.000009/0.03)
            lapspd = abs(laspeed_uav/0.03)
            lopspd = abs(lospeed_uav/0.03)
            ladiff = abs((larspd - lapspd)/larspd)
            lodiff = abs((lorspd - lopspd)/lorspd)
            
            if count_num > 1:
                ladiff_total = ladiff_total + ladiff
                lodiff_total = lodiff_total + lodiff
                ladiff_mean = ladiff_total/(count_num-1)
                lodiff_mean = lodiff_total/(count_num-1)
                print("la誤差率 = ", ladiff_mean)
                print("lo誤差率 = ", lodiff_mean)
            
            lat_position = mec.C.Y/mult
            lon_position = mec.C.X/mult
            print("finish")
            
        end = time.time()
        t = end - start
        if t < 0.03:
            time.sleep(0.03-t)

Current users number =  100
Out of range
Kicked off user below  98
Kick off user  0
Kick off user  1
Kick off user  2
Kick off user  3
Kick off user  4
Kick off user  5
Kick off user  6
Kick off user  7
Kick off user  8
Kick off user  9
Kick off user  10
Kick off user  11
Kick off user  12
Kick off user  13
Kick off user  14
Kick off user  16
Kick off user  17
Kick off user  18
Kick off user  19
Kick off user  20
Kick off user  21
Kick off user  22
Kick off user  23
Kick off user  24
Kick off user  25
Kick off user  26
Kick off user  27
Kick off user  28
Kick off user  29
Kick off user  30
Kick off user  31
Kick off user  32
Kick off user  33
Kick off user  34
Kick off user  35
Kick off user  36
Kick off user  37
Kick off user  38
Kick off user  39
Kick off user  40
Kick off user  41
Kick off user  42
Kick off user  43
Kick off user  44
Kick off user  45
Kick off user  46
Kick off user  47
Kick off user  48
Kick off user  49
Kick off user  50
Kick off user  51
Kick off user  52
Kick of

TypeError: unsupported operand type(s) for /: 'str' and 'float'